<a href="https://colab.research.google.com/github/SamwelJane/WEEK-7-IP/blob/master/WEEK_7_IP_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DEFINING THE QUESTION

##   SPECIFYING THE QUESTION:
* Make Prediction result of a game between team 1 and team 2, based on who's home and who's away, and on whether or not the game is friendly (include rank ing).

### Approach 1: Polynomial approach

* Model 1: Predict how many goals the home team scores

* Model 2: Predict how many goals the away team scores

### Approach 2 Logistic approach
Figure out from the home team’s perspective if the game is a Win, Lose or Draw (W, L, D)

**bold text**# Metrics for Success
* Perform a model with  high accuracy
*Optimize my Model using  Cross Validation
*Perform RMSE

# Context

A points system is used, with points being awarded based on the results of all FIFA-recognised full international matches.
The points which are added or subtracted are partially determined by the relative strength of the two opponents, including the logical expectation that teams higher in the ranking should fare better against teams lower in the ranking.In this analysis I shall be predicting which team is likely to be rank the first based on a series of how ranking have been taking place.

# Experimental Designs

Perform your EDA
Perform  necessary feature engineering
Check of multicollinearity
Building a model

* Approach 1: Polynomial regression model

* Approach 2: Logistic regression model

* Approach 3: Ridge, Lasso, & Elastic Net Regression models

Predict the rank of the teams and compare the models.

Perform  RMSE
Create residual plots for the model

Challenge the solution.



In [0]:
# Importing Libraries we use for our analysis

import pandas as pd
import numpy as np
import scipy as sp

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline


from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, recall_score, precision_score
from sklearn.metrics import mean_squared_error,  r2_score 
from sklearn.model_selection import KFold, LeaveOneOut
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics

import warnings
warnings.filterwarnings('ignore')

In [0]:
# loading our dataset and preview
#  fifa_ranking and results datasets
rank = pd.read_csv('fifa_ranking.csv')
results = pd.read_csv('results.csv')
 
  

In [0]:
# preview  first 5 row in both dataframes
rank.head(2)

,rank,country_full,country_abrv,total_points,previous_points,rank_change,cur_year_avg,cur_year_avg_weighted,last_year_avg,last_year_avg_weighted,two_year_ago_avg,two_year_ago_weighted,three_year_ago_avg,three_year_ago_weighted,confederation,rank_date
0,1,Germany,GER,0.0,57,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,UEFA,1993-08-08
1,2,Italy,ITA,0.0,57,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,UEFA,1993-08-08


In [0]:
results.head(2)

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
0,1872-11-30,Scotland,England,0,0,Friendly,Glasgow,Scotland,False
1,1873-03-08,England,Scotland,4,2,Friendly,London,England,False


# Exploratory Data Analysis

In [0]:
# Check the  information  overview of both dataframes
print('rank dataframe',rank.info())

print('/n*********************************')

print('resultsdataframe:',results.info())

 # The first dataframe which is rank has 57793 rows  and 16 columns while results  has 40839 rows and 9 columns 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57793 entries, 0 to 57792
Data columns (total 16 columns):
rank                       57793 non-null int64
country_full               57793 non-null object
country_abrv               57793 non-null object
total_points               57793 non-null float64
previous_points            57793 non-null int64
rank_change                57793 non-null int64
cur_year_avg               57793 non-null float64
cur_year_avg_weighted      57793 non-null float64
last_year_avg              57793 non-null float64
last_year_avg_weighted     57793 non-null float64
two_year_ago_avg           57793 non-null float64
two_year_ago_weighted      57793 non-null float64
three_year_ago_avg         57793 non-null float64
three_year_ago_weighted    57793 non-null float64
confederation              57793 non-null object
rank_date                  57793 non-null object
dtypes: float64(9), int64(3), object(4)
memory usage: 7.1+ MB
rank dataframe None
/n*****************

In [0]:
# check for null values in each dataframe

print('rank dataframe',rank.isnull().sum())

print('/n*********************************')

print('resultsdataframe:',results.isnull().sum())
#  both dataframe are  clean since they  have no missing values


rank dataframe rank                       0
country_full               0
country_abrv               0
total_points               0
previous_points            0
rank_change                0
cur_year_avg               0
cur_year_avg_weighted      0
last_year_avg              0
last_year_avg_weighted     0
two_year_ago_avg           0
two_year_ago_weighted      0
three_year_ago_avg         0
three_year_ago_weighted    0
confederation              0
rank_date                  0
dtype: int64
/n*********************************
resultsdataframe: date          0
home_team     0
away_team     0
home_score    0
away_score    0
tournament    0
city          0
country       0
neutral       0
dtype: int64


In [0]:
# Checking a summary report our dataframes : ranking and results 
import pandas_profiling as pp
rank_report = pp.ProfileReport(rank)
rank_report

Number of variables,16
Number of observations,57793
Total Missing (%),0.0%
Total size in memory,7.1 MiB
Average record size in memory,128.0 B
Numeric,4
Categorical,4
Boolean,0
Date,0
Text (Unique),0
Rejected,8


In [0]:
results_report = pp.ProfileReport(results)
results_report

# Feature Engineering

In [0]:
# creating a  column for score difference between  home_score and  away_score 
# which will help us in knownin which team  won of whether ther hasd a draw

results['score'] = results['home_score'] - results['away_score']


In [0]:
results.head()

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,score
0,1872-11-30,Scotland,England,0,0,Friendly,Glasgow,Scotland,False,0
1,1873-03-08,England,Scotland,4,2,Friendly,London,England,False,2
2,1874-03-07,Scotland,England,2,1,Friendly,Glasgow,Scotland,False,1
3,1875-03-06,England,Scotland,2,2,Friendly,London,England,False,0
4,1876-03-04,Scotland,England,3,0,Friendly,Glasgow,Scotland,False,3


In [0]:
# creating copies of my dataframes 
results_1 = results.copy()
results_1.head()

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,score
0,1872-11-30,Scotland,England,0,0,Friendly,Glasgow,Scotland,False,0
1,1873-03-08,England,Scotland,4,2,Friendly,London,England,False,2
2,1874-03-07,Scotland,England,2,1,Friendly,Glasgow,Scotland,False,1
3,1875-03-06,England,Scotland,2,2,Friendly,London,England,False,0
4,1876-03-04,Scotland,England,3,0,Friendly,Glasgow,Scotland,False,3


In [0]:
#  creatin  a copy for ranks which will be called rank_1
rank_1 = rank.copy()
rank_1.head()

,rank,country_full,country_abrv,total_points,previous_points,rank_change,cur_year_avg,cur_year_avg_weighted,last_year_avg,last_year_avg_weighted,two_year_ago_avg,two_year_ago_weighted,three_year_ago_avg,three_year_ago_weighted,confederation,rank_date
0,1,Germany,GER,0.0,57,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,UEFA,1993-08-08
1,2,Italy,ITA,0.0,57,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,UEFA,1993-08-08
2,3,Switzerland,SUI,0.0,50,9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,UEFA,1993-08-08
3,4,Sweden,SWE,0.0,55,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,UEFA,1993-08-08
4,5,Argentina,ARG,0.0,51,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CONMEBOL,1993-08-08


In [0]:
#  creating a function that  shows from the score difference that the game was either win,draw or loss.
# the score is for team

def home_team(goals):
    if goals > 0:
        return 'win'
    elif goals < 0:
        return 'loss'
    else:
        return 'draw'
      
results_1['game'] = results_1['score'].apply(lambda j: home_team(j))

In [0]:
#  creating a function that  shows from the score difference that the game was either win,draw or loss.
# the score is for team

def away_team(goals):
    if goals < 0:
        return 'win'
    elif goals > 0:
        return 'loss'
    else:
        return 'draw'
      
results_1['game'] = results_1['score'].apply(lambda j: away_team(j))

# both functions create a column called game which shows whether a away_team or home_team win,lost or drew in any given match

In [0]:
results_1.head()

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,score,game
0,1872-11-30,Scotland,England,0,0,Friendly,Glasgow,Scotland,False,0,draw
1,1873-03-08,England,Scotland,4,2,Friendly,London,England,False,2,loss
2,1874-03-07,Scotland,England,2,1,Friendly,Glasgow,Scotland,False,1,loss
3,1875-03-06,England,Scotland,2,2,Friendly,London,England,False,0,draw
4,1876-03-04,Scotland,England,3,0,Friendly,Glasgow,Scotland,False,3,loss


In [0]:
#convert column date into datetime format
results_1['year'] = pd.DatetimeIndex(results_1['date']).year
results_1['month'] = pd.DatetimeIndex(results_1['date']).month


rank_1['year'] = pd.DatetimeIndex(rank_1['rank_date']).year
rank_1['month'] = pd.DatetimeIndex(rank_1['rank_date']).month

In [0]:
results_1.head()

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,score,game,year,month
0,1872-11-30,Scotland,England,0,0,Friendly,Glasgow,Scotland,False,0,draw,1872,11
1,1873-03-08,England,Scotland,4,2,Friendly,London,England,False,2,loss,1873,3
2,1874-03-07,Scotland,England,2,1,Friendly,Glasgow,Scotland,False,1,loss,1874,3
3,1875-03-06,England,Scotland,2,2,Friendly,London,England,False,0,draw,1875,3
4,1876-03-04,Scotland,England,3,0,Friendly,Glasgow,Scotland,False,3,loss,1876,3


In [0]:
# The next function will be  grouping tournament into three categories, that is frindly,world cup and the rest grouped as others
def match(typ):
    if typ == 'Friendly':
        return 'Friendly'
    elif typ == 'FIFA World Cup':
        return 'World cup'
    else:
        return 'Other'
      
results_1['Matches'] = results_1.tournament.apply(lambda x: match(x))
#  The classification helps us to know which type of game was played by any given team

In [0]:
results_1.head()

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,score,game,year,month,Matches
0,1872-11-30,Scotland,England,0,0,Friendly,Glasgow,Scotland,False,0,draw,1872,11,Friendly
1,1873-03-08,England,Scotland,4,2,Friendly,London,England,False,2,loss,1873,3,Friendly
2,1874-03-07,Scotland,England,2,1,Friendly,Glasgow,Scotland,False,1,loss,1874,3,Friendly
3,1875-03-06,England,Scotland,2,2,Friendly,London,England,False,0,draw,1875,3,Friendly
4,1876-03-04,Scotland,England,3,0,Friendly,Glasgow,Scotland,False,3,loss,1876,3,Friendly


In [0]:
# creating a new data frame for ranking called rank_2 whcich has variables that I think might be important for this analysis
rank_2 = rank_1[['rank','country_full','total_points','previous_points','rank_date','year','month']]
rank_2.head()

,rank,country_full,total_points,previous_points,rank_date,year,month
0,1,Germany,0.0,57,1993-08-08,1993,8
1,2,Italy,0.0,57,1993-08-08,1993,8
2,3,Switzerland,0.0,50,1993-08-08,1993,8
3,4,Sweden,0.0,55,1993-08-08,1993,8
4,5,Argentina,0.0,51,1993-08-08,1993,8


In [0]:
results_2 = results_1[['home_team', 'away_team','home_score', 'away_score','Matches', 'country','score','neutral','game','year','month']]
results_2.head()

,home_team,away_team,home_score,away_score,Matches,country,score,neutral,game,year,month
0,Scotland,England,0,0,Friendly,Scotland,0,False,draw,1872,11
1,England,Scotland,4,2,Friendly,England,2,False,loss,1873,3
2,Scotland,England,2,1,Friendly,Scotland,1,False,loss,1874,3
3,England,Scotland,2,2,Friendly,England,0,False,draw,1875,3
4,Scotland,England,3,0,Friendly,Scotland,3,False,loss,1876,3


In [0]:
rank_3 = rank_2.rename({'country_full': 'home_team'}, axis = 1)
rank_3.head()

,rank,home_team,total_points,previous_points,rank_date,year,month
0,1,Germany,0.0,57,1993-08-08,1993,8
1,2,Italy,0.0,57,1993-08-08,1993,8
2,3,Switzerland,0.0,50,1993-08-08,1993,8
3,4,Sweden,0.0,55,1993-08-08,1993,8
4,5,Argentina,0.0,51,1993-08-08,1993,8


# Merging  DataFrames

In [0]:
#  merging the two dataframes: results_2 and rank_3 using a left join
df = pd.merge(results_2,rank_3, how="left", on = ['home_team', 'year', 'month'])

In [0]:
# preview the last five rows of our new dataframe
df.tail()

,home_team,away_team,home_score,away_score,Matches,country,score,neutral,game,year,month,rank,total_points,previous_points,rank_date
40903,American Samoa,Tahiti,8,1,Other,Samoa,7,True,loss,2019,7,NaN,NaN,NaN,NaN
40904,Fiji,Solomon Islands,4,4,Other,Samoa,0,True,draw,2019,7,NaN,NaN,NaN,NaN
40905,Senegal,Algeria,0,1,Other,Egypt,-1,True,win,2019,7,NaN,NaN,NaN,NaN
40906,Tajikistan,North Korea,0,1,Other,India,-1,True,win,2019,7,NaN,NaN,NaN,NaN
40907,Papua New Guinea,Fiji,1,1,Other,Samoa,0,True,draw,2019,7,NaN,NaN,NaN,NaN


# b.Exploratory Data Analysis

This section come in after we have merged thw datasets to form ne dataset called df

In [0]:
#  checking the shape of our datast df

df.shape
#  it has 18553 rowa and 15 columns


(18553, 15)

In [0]:
#checking information about our 
# info also provide us with information about the datatypes we are working with under columns
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40908 entries, 0 to 40907
Data columns (total 15 columns):
home_team          40908 non-null object
away_team          40908 non-null object
home_score         40908 non-null int64
away_score         40908 non-null int64
Matches            40908 non-null object
country            40908 non-null object
score              40908 non-null int64
neutral            40908 non-null bool
game               40908 non-null object
year               40908 non-null int64
month              40908 non-null int64
rank               18609 non-null float64
total_points       18609 non-null float64
previous_points    18609 non-null float64
rank_date          18609 non-null object
dtypes: bool(1), float64(3), int64(5), object(6)
memory usage: 4.7+ MB


In [0]:
#  checking the sum of null values in each column: rank,total_points,previous_point and rank_dates have null values
df.isnull().sum()

home_team              0
away_team              0
home_score             0
away_score             0
Matches                0
country                0
score                  0
neutral                0
game                   0
year                   0
month                  0
rank               22299
total_points       22299
previous_points    22299
rank_date          22299
dtype: int64

In [0]:
# The reason why we have null values is because in results_2 dataframe results were captured as from 1872 while ranking started in 1993
# # We are also not provided with rankig dates for 2019
# I decided to drop the null values
df.dropna(inplace = True)

In [0]:
df.isnull().sum()

home_team          0
away_team          0
home_score         0
away_score         0
Matches            0
country            0
score              0
neutral            0
game               0
year               0
month              0
rank               0
total_points       0
previous_points    0
rank_date          0
dtype: int64

In [0]:
# checking for Outliers

column=['home_score','away_score']
plt.subplots(figsize=(8,8))
df.boxplot('home_score')
plt.title=('Outliers for home score and away score')
plt.show()
#Both Home score and away score  have outliers which rangaes between  4 - 32 in home score and 6-22 in away score 

In [0]:

plt.figure(figsize = (7,8), dpi = 100)
sns.countplot(df['Matches'])
plt.xlabel('Status (draw = 0, lose =1, win = 2)')
plt.ylabel('Count')
plt.title('A Countplot of Home Team Match Outcome', color = 'b')

# Multicollinearity

In [0]:
#Before we build a model we  will  check how  the independent variables are correlated to each other

# Getting the pearson correation coefficient for each of the variables

correlation = df.corr()
correlation
# In all variables that I had in my final dataset (df) there were no variables that a collinearity of more than 1
# meaning they fit to be used as independent variables(x) since in relation to the dependend varaible(y)

,home_score,away_score,score,neutral,year,month,rank,total_points,previous_points
home_score,1.000000,-0.160455,0.831314,-0.036547,-0.039107,0.020935,-0.111440,0.023678,0.072797
away_score,-0.160455,1.000000,-0.681991,0.094300,-0.008711,0.016931,0.216275,-0.060448,-0.129112
score,0.831314,-0.681991,1.000000,-0.080180,-0.024072,0.005979,-0.204357,0.051583,0.126643
neutral,-0.036547,0.094300,-0.080180,1.000000,0.041464,-0.102805,0.045069,0.030617,-0.011351
year,-0.039107,-0.008711,-0.024072,0.041464,1.000000,0.002393,0.105348,0.618924,0.383541
month,0.020935,0.016931,0.005979,-0.102805,0.002393,1.000000,0.033436,0.030811,-0.046648
rank,-0.111440,0.216275,-0.204357,0.045069,0.105348,0.033436,1.000000,-0.292595,-0.665406
total_points,0.023678,-0.060448,0.051583,0.030617,0.618924,0.030811,-0.292595,1.000000,0.526569
previous_points,0.072797,-0.129112,0.126643,-0.011351,0.383541,-0.046648,-0.665406,0.526569,1.000000


In [0]:
#  checking how many duplictes we have in a our new dataset
df.duplicated().sum()
#  the new dataset has 56 duplicates

56

In [0]:
# dropping the duplicates
df.drop_duplicates(inplace = True)

In [0]:
# checking whether the duplicates have dropped
df.duplicated().sum()

0

In [0]:
# checking for Outliers

column=['home_score','away_score']
plt.subplots(figsize=(8,8))
df.boxplot('home_score')
plt.title=('Outliers for home score and away score')
plt.show()
#Both Home score and away score  have outliers which rangaes between  4 - 32 in home score and 6-22 in away score 

# Modelling

In [0]:
#  Model 1
#  creating a column that gas x and y
X = df['home_score'].values.reshape(-1,1)
y = df['rank'].values.reshape(-1,1)